<a href="https://colab.research.google.com/github/rarizzt/IR_Project/blob/main/Preprocessing_TKI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

from matplotlib import pyplot as plt

import pandas as pd

import numpy as np

import seaborn as sns

from google.colab import drive

drive.mount('/content/drive')



df = pd.read_csv('/content/drive/MyDrive/TKI/dataset_ir_250_sampel.csv')

In [ ]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.3 MB/s eta 0:00:00


In [ ]:
# --- SEL 2: CLEANING DASAR ---
import re
import string

df['title'] = df['title'].fillna('')
df['content_snippet'] = df['content_snippet'].fillna('')

df['text_combined'] = df['title'] + " " + df['content_snippet']

def basic_cleaning(text):
    text = str(text).lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text

print("Sedang membersihkan simbol dan angka...")
df['clean_text'] = df['text_combined'].apply(basic_cleaning)

print("Contoh hasil cleaning:")
print(df['clean_text'].head(1).values)

Sedang membersihkan simbol dan angka...
Contoh hasil cleaning:
['qatar airways rute balidoha batal terbang malam ini imbas iranisrael qatar airways membatalkan penerbangan internasional rute denpasardoha yang direncanakan terbang malam ini pukul wita dari bandara internasional i gusti ngurah rai bali']


In [ ]:
# --- SEL 3: STOPWORD REMOVAL ---
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory_stop = StopWordRemoverFactory()
stopword_remover = factory_stop.create_stop_word_remover()

def remove_stopwords(text):
    return stopword_remover.remove(text)

print("Sedang menghapus kata sambung (stopword)...")
df['clean_text'] = df['clean_text'].apply(remove_stopwords)

print("Selesai! Kata sambung hilang.")

Sedang menghapus kata sambung (stopword)...
Selesai! Kata sambung hilang.


In [ ]:
# --- SEL 4: STEMMING ---
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm import tqdm

factory_stem = StemmerFactory()
stemmer = factory_stem.create_stemmer()

tqdm.pandas()

print("Mulai Stemming (Merubah kata berimbuhan ke kata dasar)...")
print("Mohon tunggu, proses ini memakan waktu...")

df['clean_text'] = df['clean_text'].progress_apply(lambda x: stemmer.stem(x))

print("\nStemming Selesai!")
print("Contoh: 'memakan' ->", stemmer.stem("memakan"))

Mulai Stemming (Merubah kata berimbuhan ke kata dasar)...
Mohon tunggu, proses ini memakan waktu...


100%|██████████| 250/250 [01:47<00:00,  2.33it/s]



Stemming Selesai!
Contoh: 'memakan' -> makan


In [ ]:
# --- SEL 5: TOKENIZATION & SAVE ---
import ast
df['tokens'] = df['clean_text'].apply(lambda x: x.split())

output_path = '/content/drive/MyDrive/TKI/dataset_ir_250_preprocessed_lengkap.csv'
df.to_csv(output_path, index=False)

print("="*40)
print("PREPROCESSING SELESAI!")
print(f"File tersimpan di: {output_path}")
print("="*40)

print(df[['title', 'clean_text', 'tokens']].head())

PREPROCESSING SELESAI!
File tersimpan di: /content/drive/MyDrive/TKI/dataset_ir_250_preprocessed_lengkap.csv
                                               title  \
0  Qatar Airways Rute Bali-Doha Batal Terbang Mal...   
1  Alfamart di IIHF 2025, Dukung Visi Halal Globa...   
2  Bahlil Akui Cuan Hilirisasi Baru Dinikmati Inv...   
3  Rosan Klaim Perang Iran Vs Israel Belum Berdam...   
4              KKP: Pulau Kecil Tak Boleh Ditambang!   

                                          clean_text  \
0  qatar airways rute balidoha batal terbang mala...   
1  alfamart iihf dukung visi halal global bpjph a...   
2  bahlil aku cuan hilir baru nikmat investor per...   
3  ros klaim perang iran vs israel dampak investa...   
4  kkp pulau kecil tak tambang menteri laut ikan ...   

                                              tokens  
0  [qatar, airways, rute, balidoha, batal, terban...  
1  [alfamart, iihf, dukung, visi, halal, global, ...  
2  [bahlil, aku, cuan, hilir, baru, nikmat, inves...